# Lab 1 - Lab Assignment

Team Members:
- Achchala Deepan - 20943939
- Manjary Muruganandan - 20950662

## Small Example

In [11]:
from docplex.mp.model import Model

# Create model
mdl = Model(name="DemoDocplex")

# Variables
x = mdl.continuous_var(name="x", lb=0)
y = mdl.continuous_var(name="y", lb=0)

# Objective: Maximize x + y
mdl.maximize(x + y)

# Constraints
mdl.add_constraint(x + 2 * y <= 4, "c1")
mdl.add_constraint(4 * x + 3 * y <= 12, "c2")

# Solve
sol = mdl.solve(log_output=True)

# Output
if sol:
    print(f"Objective value: {mdl.objective_value}")
    mdl.print_solution()
else:
    print("No solution found.")

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =             3.200000
Objective value: 3.2
objective: 3.200
status: OPTIMAL_SOLUTION(2)
  x=2.400
  y=0.800


## CFPL

In [12]:
# Import the library
from docplex.mp.model import Model

# Create model
mdl = Model("CFLP")

# Define data
facilities = ['F1', 'F2', 'F3']
customers = ['C1', 'C2', 'C3', 'C4']

fixed_costs = {'F1': 1000, 'F2': 1200, 'F3': 1100}
capacities = {'F1': 58, 'F2': 68, 'F3': 40}
demands = {'C1': 10, 'C2': 20, 'C3': 15, 'C4': 25}

transport_cost = {
    ('F1', 'C1'): 4, ('F1', 'C2'): 6, ('F1', 'C3'): 9, ('F1', 'C4'): 5,
    ('F2', 'C1'): 5, ('F2', 'C2'): 4, ('F2', 'C3'): 7, ('F2', 'C4'): 6,
    ('F3', 'C1'): 6, ('F3', 'C2'): 3, ('F3', 'C3'): 4, ('F3', 'C4'): 7,
}

# Decision variables
open_fac = mdl.binary_var_dict(facilities, name="z")
ship = mdl.continuous_var_dict(transport_cost, name="x")

# Constraints
# 1. Demand must be met
for j in customers:
    mdl.add_constraint(
        mdl.sum(ship[i, j] for i in facilities) == demands[j],
        f"demand_{j}"
    )

# 2. Capacity constraints
for i in facilities:
    mdl.add_constraint(
        mdl.sum(ship[i, j] for j in customers) <= capacities[i] * open_fac[i],
        f"capacity_{i}"
    )

# Objective: Minimize fixed cost + transport cost
mdl.minimize(
    mdl.sum(fixed_costs[f] * open_fac[f] for f in facilities) +
    mdl.sum(transport_cost[i, j] * ship[i, j] for (i, j) in transport_cost)
)

# Solve the model
solution = mdl.solve(log_output=True)

# Output results
if solution:
    print("Total cost:", mdl.objective_value)

    for f in facilities:
        if open_fac[f].solution_value > 0.5:
            print(f"{f} is open.")

    for (i, j) in transport_cost:
        flow = ship[i, j].solution_value
        if flow > 1e-6:  # To ignore tiny rounding errors
            print(f"Ship {flow:.2f} units from {i} to {j}")
else:
    print("No solution found.")

# Show the model formulation
print("\nMODEL FORMULATION:")
print(mdl.export_as_lp_string())

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced MIP has 7 rows, 15 columns, and 27 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Found incumbent of value 3630.000000 after 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 7 rows, 15 columns, and 27 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.01 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                         3630.0000      